In [1]:
import sys
import importlib
import subprocess
import math
import torch


def install_if_colab():
    if "google.colab" in sys.modules:
        print("Running in Google Colab. Checking required libraries...")

        packages = ["moabb", "braindecode"]  # Add required libraries
        missing_packages = [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]

        if missing_packages:
            print(f"Installing missing libraries: {', '.join(missing_packages)}")
            !pip install {" ".join(missing_packages)}
        else:
            print("All required libraries are already installed.")
    else:
        print("Not running in Google Colab. No installation needed.")

install_if_colab()


Running in Google Colab. Checking required libraries...
All required libraries are already installed.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
train_set = torch.load(path+'EEG_data/emotion_train_set.pt')
test_set = torch.load(path+'EEG_data/emotion_test_set.pt')

In [4]:
print(train_set[0])

(tensor([[ 0.9150,  0.8747,  0.5803,  ..., -0.2005, -0.1347, -0.3848],
        [ 1.7004,  1.6020,  1.6953,  ...,  1.0995,  0.8411,  0.8266],
        [ 0.7192,  0.8144,  0.8551,  ...,  0.2869,  0.0526, -0.1273],
        ...,
        [-0.8671, -0.8598, -0.8854,  ..., -0.8429, -0.8595, -0.9021],
        [-0.6019, -0.5692, -0.6069,  ..., -0.5514, -0.5329, -0.5943],
        [-1.3706, -1.3457, -1.3528,  ..., -1.3049, -1.3285, -1.3405]]), 2)


In [5]:
print(test_set[0])

(tensor([[-0.5158, -0.6248, -0.6551,  ..., -0.4628, -0.5278, -0.4867],
        [-1.2126, -1.2868, -1.2099,  ..., -1.5684, -1.4594, -1.3268],
        [-0.8480, -0.9666, -1.3216,  ..., -1.5301, -1.5275, -1.4339],
        ...,
        [-0.4385, -0.4423, -0.4248,  ..., -1.4230, -1.4142, -1.4046],
        [-0.7436, -0.7013, -0.6265,  ..., -1.3699, -1.3475, -1.3313],
        [-0.1592, -0.1750, -0.1235,  ..., -0.8981, -0.8791, -0.8352]]), 2)


In [6]:
import torch
#from shallow_fbcsp import ShallowFBCSPNet
from braindecode.util import set_random_seeds


cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 2938475
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 3
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_channels = 32
input_window_samples = 400

print("n_classes: ", n_classes)
print("n_channels:", n_channels)
print("input_window_samples size:", input_window_samples)

n_classes:  3
n_channels: 32
input_window_samples size: 400


/usr/local/lib/python3.11/dist-packages/braindecode/util.py:51: UserWarning: torch.backends.cudnn.benchmark was set to True which may results in lack of reproducibility. In some cases to ensure reproducibility you may need to set torch.backends.cudnn.benchmark to False.
  warn(


In [7]:
#from models_fbscp import CollapsedShallowNet
# The ShallowFBCSPNet is a `nn.Sequential` model
import importlib
import Spatial_attention_model
importlib.reload(Spatial_attention_model)
from Spatial_attention_model import ShallowAttentionNet
model = ShallowAttentionNet(
    n_chans=n_channels,
    n_outputs=n_classes,
    n_times=input_window_samples,
)

# Display torchinfo table describing the model
print(model)

# Send model to GPU
if cuda:
    model.cuda()



ShallowAttentionNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (spatial_att): SpatialAttention(
    (conv): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (pool): MaxPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=960, out_features=3, bias=True)
)


In [8]:
# splitted = windows_dataset.split("session")
# train_set = splitted['0train']  # Session train
# test_set = splitted['1test']  # Session evaluation

from torch.nn import Module
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader

#lr = 1e-4
#weight_decay = 1e-4
#batch_size = 64
#n_epochs = 200


In [9]:

from tqdm import tqdm
# Define a method for training one epoch


def train_one_epoch(
        dataloader: DataLoader, model: Module, loss_fn, optimizer,
        scheduler: LRScheduler, epoch: int, device, print_batch_stats=True
):
    model.train()  # Set the model to training mode
    train_loss, correct = 0, 0

    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader),
                        disable=not print_batch_stats)

    for batch_idx, (X, y) in progress_bar:
        X, y = X.to(device), y.to(device)
        #print(X.shape)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()  # update the model weights
        optimizer.zero_grad()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).sum().item()

        #if print_batch_stats:
        #    progress_bar.set_description(
        #        f"Epoch {epoch}/{n_epochs}, "
        #        f"Batch {batch_idx + 1}/{len(dataloader)}, "
        #        f"Loss: {loss.item():.6f}"
        #    )

    # Update the learning rate
    scheduler.step()

    correct /= len(dataloader.dataset)
    return train_loss / len(dataloader), correct


In [10]:
from collections import defaultdict
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np

@torch.no_grad()
def test_model(dataloader: DataLoader, model: torch.nn.Module, loss_fn, print_batch_stats=True):
    device = next(model.parameters()).device  # Get model device
    size = len(dataloader.dataset)
    n_batches = len(dataloader)
    model.eval()  # Switch to evaluation mode
    test_loss, correct = 0, 0

    # Initialize dictionaries for per-class tracking
    class_correct = defaultdict(int)
    class_total = defaultdict(int)

    # Lists to store true and predicted labels for confusion matrix
    all_preds = []
    all_targets = []

    if print_batch_stats:
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    else:
        progress_bar = enumerate(dataloader)

    for batch_idx, (X, y) in progress_bar:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        batch_loss = loss_fn(pred, y).item()

        test_loss += batch_loss
        correct += (pred.argmax(1) == y).sum().item()

        # Store predictions and true labels for confusion matrix
        all_preds.append(pred.argmax(1).cpu())
        all_targets.append(y.cpu())

        # Compute per-class accuracy
        preds_labels = pred.argmax(1)
        for label, pred_label in zip(y, preds_labels):
            class_total[label.item()] += 1
            class_correct[label.item()] += (label == pred_label).item()

        if print_batch_stats:
            progress_bar.set_description(
                f"Batch {batch_idx + 1}/{len(dataloader)}, Loss: {batch_loss:.6f}"
            )

    # Convert lists to tensors
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)

    # Compute per-class accuracy
    class_accuracies = {
        cls: (class_correct[cls] / class_total[cls]) * 100 if class_total[cls] > 0 else 0
        for cls in class_total
    }

    # Compute overall accuracy
    test_loss /= n_batches
    overall_accuracy = (correct / size) * 100

    # Print per-class accuracy
    print("\nClass-wise Accuracy:")
    for cls, acc in class_accuracies.items():
        print(f"  Class {cls}: {acc:.2f}%")

    print(f"Test Accuracy: {overall_accuracy:.1f}%, Test Loss: {test_loss:.6f}\n")

    return test_loss, overall_accuracy, class_accuracies, all_preds, all_targets


In [11]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ahmm98 (philinthesky) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
import torch
import wandb
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import CrossEntropyLoss
import numpy as np
from weight_init import init_weights

seeds = [99999,182726,91111222,44552222,12223111,100300,47456655,4788347,77766666,809890]
for _seed in seeds:
  seed = _seed
  set_random_seeds(seed=seed, cuda=cuda)
  model = ShallowAttentionNet(
    n_chans=n_channels,
    n_outputs=n_classes,
    n_times=input_window_samples,
  )

  # Display torchinfo table describing the model
  print(model)
  model.apply(init_weights)
  # Send model to GPU
  if cuda:
      model.cuda()
  # Initialize Weights & Biases
  wandb.init(project="Master Thesis", name=f"{model.__class__.__name__} {seed}")

  # Define hyperparameters
  lr = 1e-3
  weight_decay = 1e-4
  batch_size = 32  # Start with 124
  n_epochs = 60
  patience = 30  # Patience for early stopping

  final_acc = 0.0

  # Log hyperparameters to wandb
  wandb.config.update({
      "learning_rate": lr,
      "weight_decay": weight_decay,
      "batch_size": batch_size,
      "epochs": n_epochs
  })

  # Define optimizer and scheduler
  optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs - 1)

  # Define loss function
  loss_fn = CrossEntropyLoss()

  # Create DataLoaders
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_set, batch_size=batch_size)

  # Initialize lists to store all predictions & targets
  all_preds, all_targets = [], []

  # Early Stopping setup
  best_val_loss = float('inf')
  epochs_without_improvement = 0

  # Training loop
  for epoch in range(1, n_epochs + 1):
      print(f"Epoch {epoch}/{n_epochs}: ", end="")

      train_loss, train_accuracy = train_one_epoch(
          train_loader, model, loss_fn, optimizer, scheduler, epoch, device
      )

      test_loss, test_accuracy, class_accuracies, batch_preds, batch_targets = test_model(test_loader, model, loss_fn)
      final_acc = test_accuracy

      # Store predictions & labels for confusion matrix
      all_preds.extend(batch_preds)
      all_targets.extend(batch_targets)

      # Print class-wise accuracy
      print("\nClass-wise Accuracy:")
      for class_idx, acc in class_accuracies.items():
          print(f"  Class {class_idx}: {acc:.2f}%")

      # Log results to wandb
      wandb.log({
          "epoch": epoch,
          "train_loss": train_loss,
          "train_accuracy": train_accuracy * 100,
          "test_loss": test_loss,
          "test_accuracy": test_accuracy,
          "learning_rate": scheduler.get_last_lr()[0],
          **{f"class_{class_idx}_accuracy": acc for class_idx, acc in class_accuracies.items()}
      })

      print(
          f"Train Accuracy: {100 * train_accuracy:.2f}%, "
          f"Average Train Loss: {train_loss:.6f}, "
          f"Test Accuracy: {test_accuracy:.2f}%, "
          f"Average Test Loss: {test_loss:.6f}\n"
      )

      # Early stopping check
      if test_loss < best_val_loss:
          best_val_loss = test_loss
          epochs_without_improvement = 0  # Reset counter if we have improvement
      else:
          epochs_without_improvement += 1

      # If no improvement for 'patience' epochs, stop training early
      if epochs_without_improvement >= patience:
          print(f"Early stopping triggered after {epoch+1} epochs.")
          break

  # Convert lists to NumPy arrays
  all_preds = np.array(all_preds)
  all_targets = np.array(all_targets)

  # Save predictions & true labels for later use (confusion matrix)
  wandb.log({"all_preds": all_preds.tolist(), "all_targets": all_targets.tolist()})
  wandb.finish()
  torch.save(model, f"{model.__class__.__name__}_{math.ceil(final_acc)}_{seed}.pth")
  torch.save(model.state_dict(), f"{model.__class__.__name__}_{math.ceil(final_acc)}_{seed}_state.pth")



/usr/local/lib/python3.11/dist-packages/braindecode/util.py:51: UserWarning: torch.backends.cudnn.benchmark was set to True which may results in lack of reproducibility. In some cases to ensure reproducibility you may need to set torch.backends.cudnn.benchmark to False.
  warn(


ShallowAttentionNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (spatial_att): SpatialAttention(
    (conv): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (pool): MaxPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=960, out_features=3, bias=True)
)


Epoch 1/60: 

Batch 699/699, Loss: 1.083459: 100%|██████████| 699/699 [00:03<00:00, 197.55it/s]



Class-wise Accuracy:
  Class 2: 1.81%
  Class 0: 1.03%
  Class 1: 99.29%
Test Accuracy: 33.6%, Test Loss: 1.101808


Class-wise Accuracy:
  Class 2: 1.81%
  Class 0: 1.03%
  Class 1: 99.29%
Train Accuracy: 56.06%, Average Train Loss: 0.944016, Test Accuracy: 33.60%, Average Test Loss: 1.101808

Epoch 2/60: 

...

Batch 699/699, Loss: 0.531474: 100%|██████████| 699/699 [00:03<00:00, 209.85it/s]



Class-wise Accuracy:
  Class 2: 74.90%
  Class 0: 55.63%
  Class 1: 78.01%
Test Accuracy: 69.5%, Test Loss: 0.690070


Class-wise Accuracy:
  Class 2: 74.90%
  Class 0: 55.63%
  Class 1: 78.01%
Train Accuracy: 84.28%, Average Train Loss: 0.394322, Test Accuracy: 69.50%, Average Test Loss: 0.690070

Epoch 60/60: 

Batch 699/699, Loss: 0.630302: 100%|██████████| 699/699 [00:03<00:00, 197.70it/s]



Class-wise Accuracy:
  Class 2: 63.85%
  Class 0: 41.71%
  Class 1: 76.18%
Test Accuracy: 60.5%, Test Loss: 0.841513


Class-wise Accuracy:
  Class 2: 63.85%
  Class 0: 41.71%
  Class 1: 76.18%
Train Accuracy: 84.39%, Average Train Loss: 0.392341, Test Accuracy: 60.52%, Average Test Loss: 0.841513



class_0_accuracy,▁▁▁▂▂▁▂▅▅▂▆▅▂▆▆▅▆▄▇▁▅▇▆▇▇▇▇▆█▅▆▇▇████▆█▆
class_1_accuracy,█▆██▇▇█▅▇▆▆▅▅▁▆▆▇▅▆▆▆▆▆▅▆▅▅▅▆▆▆▆▆▆▆▆▇▇▄▄
class_2_accuracy,▁▃▁▄▃▅▃▆▆▇▇▇▆▇▆▇█▇▇▇█▆██████▇▇▆███████▇▇
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
learning_rate,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
test_accuracy,▁▂▁▃▄▂▃▆▄▆▆▆▆▃▆▅▇▃▇▅▇▇█▆██▇▆▇▇█████▇██▅▅
test_loss,▆▆▆▆█▆▆▅▃▅▃▆▃▃▅▃▄▅▃▄▂▁▂▃▂▂▂▃▂▄▁▁▃▁▁▁▂▁▁▄
train_accuracy,▁▂▃▃▃▅▅▅▅▆▆▆▅▆▆▆▇▆▇▇▇▇▇▇████████████████
train_loss,█▇▆▆▇▆▅▅▄▄▄▄▃▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
class_0_accuracy,41.70915
class_1_accuracy,76.184


In [13]:
import shutil
import os
from google.colab import drive

source_dir = "./"  # Root directory where .pth files are stored
dest_dir = "/content/drive/My Drive/pth_backups/"  # Destination in Google Drive

# Ensure destination directory exists
os.makedirs(dest_dir, exist_ok=True)

# Find and move all .pth files
for file_name in os.listdir(source_dir):
    if file_name.endswith(".pth"):
        src_path = os.path.join(source_dir, file_name)
        dest_path = os.path.join(dest_dir, file_name)
        shutil.move(src_path, dest_path)
        print(f"Moved: {file_name} -> {dest_path}")

print("All .pth files have been moved to Google Drive!")

Moved: ShallowAttentionNet_90_100300.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_90_100300.pth
Moved: ShallowAttentionNet_83_12223111.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_83_12223111.pth
Moved: ShallowAttentionNet_72_99999.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_72_99999.pth
Moved: ShallowAttentionNet_80_4788347.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_80_4788347.pth
Moved: ShallowAttentionNet_60_99999.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_60_99999.pth
Moved: ShallowAttentionNet_83_77766666_state.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_83_77766666_state.pth
Moved: ShallowAttentionNet_81_44552222.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_81_44552222.pth
Moved: ShallowAttentionNet_77_47456655.pth -> /content/drive/My Drive/pth_backups/ShallowAttentionNet_77_47456655.pth
Moved: ShallowAttentionNet_81_99999.pth -> /content/drive/My D